In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [6]:
from imblearn.combine import SMOTETomek

smt= SMOTETomek(random_state= 15)
X_sm_tl, y_sm_tl= smt.fit_resample(X_train, y_train)

In [7]:
from sklearn.feature_selection import SelectKBest, f_classif

selector= SelectKBest(score_func= f_classif, k= 10)
X_sm_tl_selected= selector.fit_transform(X_sm_tl, y_sm_tl)
X_sm_tl_test_selected= selector.transform(X_test)

In [8]:
feature_names= X.columns
selected_indices= selector.get_support(indices=True)
selected_features= feature_names[selected_indices]
print("Selected Features: ", selected_features)

Selected Features:  Index(['Cholesterol', 'Family History', 'Obesity', 'Alcohol Consumption',
       'Diet', 'Sedentary Hours Per Day', 'Triglycerides',
       'Physical Activity Days Per Week', 'Sleep Hours Per Day', 'HBP'],
      dtype='object')


--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)
lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.63      0.49      0.55      1125
           1       0.35      0.48      0.40       628

    accuracy                           0.49      1753
   macro avg       0.49      0.49      0.48      1753
weighted avg       0.53      0.49      0.50      1753

Precision: 0.34597701149425286
Accuracy: 0.48887621220764405
AUC: 0.4820396319886765


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)
dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.65      0.62      0.63      1125
           1       0.37      0.40      0.38       628

    accuracy                           0.54      1753
   macro avg       0.51      0.51      0.51      1753
weighted avg       0.55      0.54      0.54      1753

Precision: 0.3683431952662722
Accuracy: 0.5402167712492869
AUC: 0.5084706298655344


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 20}
Best Precision Score: 0.6458913557855501


In [12]:
tuned_dt= gs_dt.best_estimator_
y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.65      0.95      0.77      1125
           1       0.45      0.07      0.12       628

    accuracy                           0.64      1753
   macro avg       0.55      0.51      0.44      1753
weighted avg       0.58      0.64      0.54      1753

Precision: 0.4479166666666667
Accuracy: 0.6360524814603536
AUC: 0.5119575371549894


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
knn.fit(X_sm_tl_selected, y_sm_tl)
y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_knn, labels= knn.classes_)
print(classification_report(y_test, y_pred_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn))

              precision    recall  f1-score   support

           0       0.63      0.53      0.58      1125
           1       0.35      0.45      0.39       628

    accuracy                           0.50      1753
   macro avg       0.49      0.49      0.49      1753
weighted avg       0.53      0.50      0.51      1753

Precision: 0.3488943488943489
Accuracy: 0.5014261266400456
AUC: 0.4870785562632697


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.6895178964255761


In [15]:
tuned_knn= gs_knn.best_estimator_
y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn, labels= tuned_knn.classes_)
print(classification_report(y_test, y_pred_tuned_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn))

              precision    recall  f1-score   support

           0       0.64      0.80      0.71      1125
           1       0.35      0.19      0.24       628

    accuracy                           0.58      1753
   macro avg       0.49      0.49      0.48      1753
weighted avg       0.53      0.58      0.54      1753

Precision: 0.34615384615384615
Accuracy: 0.5824301197946378
AUC: 0.4976702052370842


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_nb, labels= nb.classes_)
print(classification_report(y_test, y_pred_nb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb))

              precision    recall  f1-score   support

           0       0.63      0.51      0.56      1125
           1       0.34      0.46      0.39       628

    accuracy                           0.49      1753
   macro avg       0.48      0.48      0.48      1753
weighted avg       0.52      0.49      0.50      1753

Precision: 0.3420738974970203
Accuracy: 0.4905875641756988
AUC: 0.4768619957537155


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc.fit(X_sm_tl_selected, y_sm_tl)
y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_svc, labels= svc.classes_)
print(classification_report(y_test, y_pred_svc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc))

              precision    recall  f1-score   support

           0       0.63      0.72      0.67      1125
           1       0.31      0.22      0.26       628

    accuracy                           0.55      1753
   macro avg       0.47      0.47      0.47      1753
weighted avg       0.51      0.55      0.52      1753

Precision: 0.31263858093126384
Accuracy: 0.5453508271534512
AUC: 0.4888188251946214


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.64      0.85      0.73      1125
           1       0.37      0.15      0.22       628

    accuracy                           0.60      1753
   macro avg       0.51      0.50      0.48      1753
weighted avg       0.55      0.60      0.55      1753

Precision: 0.3702290076335878
Accuracy: 0.602966343411295
AUC: 0.5078697806086342


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best Precision Score: 0.7477121918219427


In [21]:
tuned_rf= gs_tuned_rf.best_estimator_
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.64      0.87      0.74      1125
           1       0.35      0.13      0.19       628

    accuracy                           0.60      1753
   macro avg       0.50      0.50      0.46      1753
weighted avg       0.54      0.60      0.54      1753

Precision: 0.3524229074889868
Accuracy: 0.6035367940673132
AUC: 0.507278839348903


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.63      0.71      0.67      1125
           1       0.34      0.27      0.30       628

    accuracy                           0.55      1753
   macro avg       0.49      0.49      0.48      1753
weighted avg       0.53      0.55      0.54      1753

Precision: 0.338
Accuracy: 0.549343981745579
AUC: 0.486617126680821


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.7485840283438058


In [25]:
tuned_ada= gs_ada.best_estimator_
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.64      0.94      0.76      1125
           1       0.31      0.05      0.09       628

    accuracy                           0.62      1753
   macro avg       0.47      0.49      0.42      1753
weighted avg       0.52      0.62      0.52      1753

Precision: 0.31
Accuracy: 0.6200798630918426
AUC: 0.4893842887473461


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.64      0.98      0.78      1125
           1       0.39      0.03      0.05       628

    accuracy                           0.64      1753
   macro avg       0.52      0.50      0.41      1753
weighted avg       0.55      0.64      0.51      1753

Precision: 0.3902439024390244
Accuracy: 0.6366229321163719
AUC: 0.48452512384996466


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.772585566910753


In [28]:
tuned_grb= gs_grb.best_estimator_
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.64      0.98      0.78      1125
           1       0.39      0.03      0.05       628

    accuracy                           0.64      1753
   macro avg       0.52      0.50      0.41      1753
weighted avg       0.55      0.64      0.51      1753

Precision: 0.3902439024390244
Accuracy: 0.6366229321163719
AUC: 0.48452512384996466


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.64      0.82      0.72      1125
           1       0.37      0.19      0.25       628

    accuracy                           0.59      1753
   macro avg       0.51      0.50      0.48      1753
weighted avg       0.54      0.59      0.55      1753

Precision: 0.3667711598746082
Accuracy: 0.5932686822589845
AUC: 0.4973460721868366


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.701964160797445
              precision    recall  f1-score   support

           0       0.65      0.91      0.76      1125
           1       0.41      0.11      0.17       628

    accuracy                           0.62      1753
   macro avg       0.53      0.51      0.46      1753
weighted avg       0.56      0.62      0.55      1753

Precision: 0.40588235294117647
Accuracy: 0.623502567027952
AUC: 0.5053262561924982


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001135 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2438
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.64      0.90      0.75      1125
           1       0.34      0.09      0.15       628

    accuracy                           0.61      1753
   macro avg       0.49      0.50      0.45      1753
weighted avg       0.53      0.61      0.53      1753

Precision: 0.3430232558139535
Accuracy: 0.6109526525955505
AUC: 0.4901316348195329


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000476 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2438
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.01, 'min_data_in_leaf': 50, 'num_leaves': 31, 'reg_alpha': 0.1}
Best Precision Score: 0.7983442076752676
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_le

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)
cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.02579
0:	learn: 0.6873501	total: 164ms	remaining: 2m 43s
1:	learn: 0.6841590	total: 170ms	remaining: 1m 24s
2:	learn: 0.6794064	total: 176ms	remaining: 58.4s
3:	learn: 0.6755079	total: 181ms	remaining: 45.1s
4:	learn: 0.6714611	total: 186ms	remaining: 37s
5:	learn: 0.6666246	total: 191ms	remaining: 31.6s
6:	learn: 0.6615790	total: 195ms	remaining: 27.7s
7:	learn: 0.6575497	total: 200ms	remaining: 24.8s
8:	learn: 0.6543895	total: 205ms	remaining: 22.6s
9:	learn: 0.6519028	total: 210ms	remaining: 20.8s
10:	learn: 0.6487389	total: 215ms	remaining: 19.3s
11:	learn: 0.6462825	total: 221ms	remaining: 18.2s
12:	learn: 0.6428735	total: 226ms	remaining: 17.2s
13:	learn: 0.6391505	total: 231ms	remaining: 16.3s
14:	learn: 0.6373477	total: 235ms	remaining: 15.5s
15:	learn: 0.6350176	total: 241ms	remaining: 14.8s
16:	learn: 0.6331455	total: 246ms	remaining: 14.2s
17:	learn: 0.6316768	total: 251ms	remaining: 13.7s
18:	learn: 0.6302023	total: 256ms	remaining: 13.2s
19:	learn: 0

192:	learn: 0.5278942	total: 1.25s	remaining: 5.23s
193:	learn: 0.5276305	total: 1.26s	remaining: 5.22s
194:	learn: 0.5271365	total: 1.26s	remaining: 5.21s
195:	learn: 0.5268920	total: 1.27s	remaining: 5.2s
196:	learn: 0.5267069	total: 1.27s	remaining: 5.19s
197:	learn: 0.5265589	total: 1.28s	remaining: 5.17s
198:	learn: 0.5262744	total: 1.28s	remaining: 5.16s
199:	learn: 0.5260596	total: 1.29s	remaining: 5.15s
200:	learn: 0.5257601	total: 1.29s	remaining: 5.14s
201:	learn: 0.5255554	total: 1.3s	remaining: 5.13s
202:	learn: 0.5250466	total: 1.3s	remaining: 5.11s
203:	learn: 0.5248669	total: 1.31s	remaining: 5.1s
204:	learn: 0.5246556	total: 1.31s	remaining: 5.09s
205:	learn: 0.5243932	total: 1.32s	remaining: 5.07s
206:	learn: 0.5240887	total: 1.32s	remaining: 5.06s
207:	learn: 0.5239127	total: 1.33s	remaining: 5.05s
208:	learn: 0.5236880	total: 1.33s	remaining: 5.04s
209:	learn: 0.5235270	total: 1.33s	remaining: 5.03s
210:	learn: 0.5233715	total: 1.34s	remaining: 5.01s
211:	learn: 0.52

371:	learn: 0.4913027	total: 2.15s	remaining: 3.63s
372:	learn: 0.4911829	total: 2.16s	remaining: 3.63s
373:	learn: 0.4910184	total: 2.16s	remaining: 3.62s
374:	learn: 0.4908350	total: 2.17s	remaining: 3.61s
375:	learn: 0.4906138	total: 2.17s	remaining: 3.6s
376:	learn: 0.4904081	total: 2.17s	remaining: 3.59s
377:	learn: 0.4902947	total: 2.18s	remaining: 3.59s
378:	learn: 0.4901630	total: 2.18s	remaining: 3.58s
379:	learn: 0.4900205	total: 2.19s	remaining: 3.57s
380:	learn: 0.4898996	total: 2.19s	remaining: 3.56s
381:	learn: 0.4897258	total: 2.2s	remaining: 3.55s
382:	learn: 0.4896059	total: 2.2s	remaining: 3.55s
383:	learn: 0.4894568	total: 2.21s	remaining: 3.54s
384:	learn: 0.4893203	total: 2.21s	remaining: 3.53s
385:	learn: 0.4891676	total: 2.22s	remaining: 3.52s
386:	learn: 0.4889522	total: 2.22s	remaining: 3.52s
387:	learn: 0.4883919	total: 2.23s	remaining: 3.51s
388:	learn: 0.4882942	total: 2.23s	remaining: 3.5s
389:	learn: 0.4881180	total: 2.23s	remaining: 3.5s
390:	learn: 0.487

556:	learn: 0.4590912	total: 3.24s	remaining: 2.57s
557:	learn: 0.4589573	total: 3.24s	remaining: 2.57s
558:	learn: 0.4588343	total: 3.25s	remaining: 2.56s
559:	learn: 0.4586583	total: 3.26s	remaining: 2.56s
560:	learn: 0.4585018	total: 3.26s	remaining: 2.55s
561:	learn: 0.4583800	total: 3.27s	remaining: 2.55s
562:	learn: 0.4582472	total: 3.28s	remaining: 2.54s
563:	learn: 0.4581616	total: 3.28s	remaining: 2.54s
564:	learn: 0.4578852	total: 3.29s	remaining: 2.53s
565:	learn: 0.4577985	total: 3.29s	remaining: 2.53s
566:	learn: 0.4576905	total: 3.3s	remaining: 2.52s
567:	learn: 0.4575262	total: 3.31s	remaining: 2.51s
568:	learn: 0.4573327	total: 3.31s	remaining: 2.51s
569:	learn: 0.4572157	total: 3.32s	remaining: 2.5s
570:	learn: 0.4570865	total: 3.32s	remaining: 2.5s
571:	learn: 0.4569372	total: 3.33s	remaining: 2.49s
572:	learn: 0.4567676	total: 3.33s	remaining: 2.48s
573:	learn: 0.4565916	total: 3.34s	remaining: 2.48s
574:	learn: 0.4564682	total: 3.35s	remaining: 2.47s
575:	learn: 0.4

715:	learn: 0.4378542	total: 4.32s	remaining: 1.71s
716:	learn: 0.4377646	total: 4.33s	remaining: 1.71s
717:	learn: 0.4376867	total: 4.34s	remaining: 1.7s
718:	learn: 0.4375501	total: 4.34s	remaining: 1.7s
719:	learn: 0.4374848	total: 4.35s	remaining: 1.69s
720:	learn: 0.4373382	total: 4.36s	remaining: 1.69s
721:	learn: 0.4372213	total: 4.37s	remaining: 1.68s
722:	learn: 0.4370945	total: 4.38s	remaining: 1.68s
723:	learn: 0.4369317	total: 4.38s	remaining: 1.67s
724:	learn: 0.4368189	total: 4.39s	remaining: 1.67s
725:	learn: 0.4367112	total: 4.4s	remaining: 1.66s
726:	learn: 0.4366284	total: 4.41s	remaining: 1.66s
727:	learn: 0.4365025	total: 4.42s	remaining: 1.65s
728:	learn: 0.4363674	total: 4.42s	remaining: 1.64s
729:	learn: 0.4363291	total: 4.43s	remaining: 1.64s
730:	learn: 0.4361347	total: 4.44s	remaining: 1.63s
731:	learn: 0.4360163	total: 4.45s	remaining: 1.63s
732:	learn: 0.4359353	total: 4.46s	remaining: 1.63s
733:	learn: 0.4358144	total: 4.47s	remaining: 1.62s
734:	learn: 0.4

876:	learn: 0.4193826	total: 5.39s	remaining: 756ms
877:	learn: 0.4192928	total: 5.39s	remaining: 750ms
878:	learn: 0.4191830	total: 5.4s	remaining: 744ms
879:	learn: 0.4191032	total: 5.41s	remaining: 737ms
880:	learn: 0.4190114	total: 5.41s	remaining: 731ms
881:	learn: 0.4188774	total: 5.41s	remaining: 724ms
882:	learn: 0.4187742	total: 5.42s	remaining: 718ms
883:	learn: 0.4186424	total: 5.42s	remaining: 712ms
884:	learn: 0.4184197	total: 5.43s	remaining: 706ms
885:	learn: 0.4182305	total: 5.43s	remaining: 699ms
886:	learn: 0.4181362	total: 5.44s	remaining: 693ms
887:	learn: 0.4180335	total: 5.44s	remaining: 686ms
888:	learn: 0.4179002	total: 5.45s	remaining: 680ms
889:	learn: 0.4178228	total: 5.45s	remaining: 674ms
890:	learn: 0.4177020	total: 5.46s	remaining: 668ms
891:	learn: 0.4176204	total: 5.47s	remaining: 662ms
892:	learn: 0.4174886	total: 5.47s	remaining: 656ms
893:	learn: 0.4173818	total: 5.48s	remaining: 650ms
894:	learn: 0.4172522	total: 5.49s	remaining: 644ms
895:	learn: 0

In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6881936	total: 2.21ms	remaining: 218ms
1:	learn: 0.6855949	total: 5.18ms	remaining: 254ms
2:	learn: 0.6833151	total: 7.88ms	remaining: 255ms
3:	learn: 0.6801449	total: 10.7ms	remaining: 256ms
4:	learn: 0.6780472	total: 12.9ms	remaining: 244ms
5:	learn: 0.6758095	total: 15.1ms	remaining: 237ms
6:	learn: 0.6735855	total: 18.1ms	remaining: 240ms
7:	learn: 0.6720314	total: 20.5ms	remaining: 235ms
8:	learn: 0.6704444	total: 23.1ms	remaining: 233ms
9:	learn: 0.6698024	total: 25.9ms	remaining: 234ms
10:	learn: 0.6687722	total: 29.8ms	remaining: 241ms
11:	learn: 0.6661773	total: 33.7ms	remaining: 247ms
12:	learn: 0.6625990	total: 36.6ms	remaining: 245ms
13:	learn: 0.6605971	total: 39.4ms	remaining: 242ms
14:	learn: 0.6583857	total: 42.6ms	remaining: 241ms
15:	learn: 0.6552257	total: 45.3ms	remaining: 238ms
16:	learn: 0.6540093	total: 47.6ms	remaining: 232ms
17:	learn: 0.6513803	total: 49.2ms	remaining: 224ms
18:	learn: 0

--- Visualisation ---

In [35]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

annova_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
annova_precision= annova_precision.sort_values(by= 'Precision Score', ascending=False)
print(annova_precision)

                                    Model  Precision Score
2          Tuned Decision Tree Precision:         0.447917
14                   Tuned XGB Precision:         0.405882
11            GradientBoosting Precision:         0.390244
12      Tuned GradientBoosting Precision:         0.390244
16                  Tuned LGBM Precision:         0.380000
7                Random Forest Precision:         0.370229
1                Decision Tree Precision:         0.368343
13                         XGB Precision:         0.366771
8          Tuned Random Forest Precision:         0.352423
3         KNeighborsClassifier Precision:         0.348894
4   Tuned KNeighborsClassifier Precision:         0.346154
0          Logistic Regression Precision:         0.345977
17                    CatBoost Precision:         0.345679
15                        LGBM Precision:         0.343023
5                   GaussianNB Precision:         0.342074
9                     AdaBoost Precision:         0.3380

In [36]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

annova_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
annova_accuracy= annova_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(annova_accuracy)

                                   Model  Accuracy Score
11            GradientBoosting Accuracy:        0.636623
12      Tuned GradientBoosting Accuracy:        0.636623
18              Tuned CatBoost Accuracy:        0.636052
2          Tuned Decision Tree Accuracy:        0.636052
16                  Tuned LGBM Accuracy:        0.634912
14                   Tuned XGB Accuracy:        0.623503
10              Tuned AdaBoost Accuracy:        0.620080
17                    CatBoost Accuracy:        0.613234
15                        LGBM Accuracy:        0.610953
8          Tuned Random Forest Accuracy:        0.603537
7                Random Forest Accuracy:        0.602966
13                         XGB Accuracy:        0.593269
4   Tuned KNeighborsClassifier Accuracy:        0.582430
9                     AdaBoost Accuracy:        0.549344
6                          SVM Accuracy:        0.545351
1                Decision Tree Accuracy:        0.540217
3         KNeighborsClassifier 

In [37]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

annova_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
annova_auc= annova_auc.sort_values(by= 'AUC Score', ascending=False)
print(annova_auc)

                              Model  AUC Score
2          Tuned Decision Tree AUC:   0.511958
1                Decision Tree AUC:   0.508471
7                Random Forest AUC:   0.507870
8          Tuned Random Forest AUC:   0.507279
14                   Tuned XGB AUC:   0.505326
17                    CatBoost AUC:   0.505205
4   Tuned KNeighborsClassifier AUC:   0.497670
13                         XGB AUC:   0.497346
15                        LGBM AUC:   0.490132
10              Tuned AdaBoost AUC:   0.489384
16                  Tuned LGBM AUC:   0.488836
6                          SVM AUC:   0.488819
3         KNeighborsClassifier AUC:   0.487079
9                     AdaBoost AUC:   0.486617
11            GradientBoosting AUC:   0.484525
12      Tuned GradientBoosting AUC:   0.484525
0          Logistic Regression AUC:   0.482040
18              Tuned CatBoost AUC:   0.481871
5                   GaussianNB AUC:   0.476862
